In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"


import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

2023-10-12 04:05:47.289394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 04:05:47.937964: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
strategy = tf.distribute.MirroredStrategy()
# Create Autoencoder
def LWAED(image_size, gray=False):
    kernel_size = 3  # (3, 3)
    pooling_size = 2  # (2, 2)

    image_shape = np.concatenate((image_size, [1] if gray else [3]))
    with strategy.scope():
        model = Sequential()

        # Input layer
        model.add(Input(shape=image_shape))

        #########
        # Encoder
        # Conv
        model.add(Conv2D(50, kernel_size=kernel_size, activation='relu', padding='same'))
        model.add(AveragePooling2D(pooling_size, padding='same'))
        model.add(Conv2D(50, kernel_size=kernel_size, activation='relu', padding='same'))
        model.add(AveragePooling2D(pooling_size, padding='same'))
        # Dense
        model.add(Dense(10))

        # Decoder
        # Dense
        model.add(Dense(10))
        # Conv
        model.add(Conv2D(50, kernel_size=kernel_size, activation='relu', padding='same'))
        model.add(UpSampling2D(pooling_size))
        model.add(Conv2D(50, kernel_size=kernel_size, activation='relu', padding='same'))
        model.add(UpSampling2D(pooling_size))

        # Output layer
        # model.add(Conv2D(3, kernel_size=kernel_size, activation='sigmoid', padding='same'))
        model.add(Conv2D(3, kernel_size=kernel_size, activation='relu', padding='same'))

        opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
        model.compile(optimizer=opt, loss='mse')
        model.build()
        model.summary()

    return model

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2023-10-12 04:05:56.794199: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22319 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:a1:00.0, compute capability: 8.6


In [ ]:
import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm

def resizer_function(image_paths, size=(360, 640)):
    num_images = len(image_paths)
    data = np.empty((num_images, size[1], size[0], 3), dtype=np.float32)

    for i, image_path in tqdm(enumerate(image_paths), total=num_images):
        img = np.array((Image.open(image_path).convert('RGB').resize(size, Image.LANCZOS)), dtype=np.float32)
        img /= 255.0 
        data[i] = img

    return data

def load_images_with_glob(directory, size, max_images=None):
    image_paths = glob.glob(os.path.join(directory, "*.jpg"))  
    image_paths.sort()
    if max_images is not None:
        image_paths = image_paths[:max_images]
    return image_paths


image_size = (640, 360)
max_images = 39000

dataset_path1 = '/media/hdd/aryan/ifseg3/Dataset/NEW_MASK_2.0'
dataset_path2 = '/media/hdd/aryan/ifseg3/Dataset/TRAIN_CUSTOM'

# Load X and y, Normalized 
x = load_images_with_glob(dataset_path1, image_size, max_images)
y = load_images_with_glob(dataset_path2, image_size, max_images)

In [ ]:
image_size = (360, 640)
with strategy.scope():
    model = LWAED(image_size=image_size)

In [ ]:
from tqdm import tqdm
import numpy as np

batch_size = 26
num_batches = len(x) // batch_size


for i in tqdm(range(num_batches), desc="Training Batches"):
    print("BATCH : ", i)
    
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size if i < num_batches - 1 else len(x)
    
    print("start_idx, end_idx : ", start_idx, end_idx)
    image_size = (640, 360)
    x_images = resizer_function(x[start_idx:end_idx], image_size)
    y_images = resizer_function(y[start_idx:end_idx], image_size)
    
    with tf.device('/device:GPU:0'):
        x_batch = tf.convert_to_tensor(x_images[0:19]) 
    with tf.device('/device:GPU:0'):
        y_batch = tf.convert_to_tensor(y_images[0:19])
    
    # Train the model on the current batch
    history = model.fit(x_batch, y_batch, validation_split=0.1, epochs=200, batch_size=batch_size, verbose=0)
    train_loss_values = history.history['loss']
    val_loss_values = history.history['val_loss']
    average_train_loss = sum(train_loss_values) / len(train_loss_values)
    average_val_loss = sum(val_loss_values) / len(val_loss_values)
    print(f"Average Validation Loss: {average_val_loss:.6f} " + f"Average Training Loss: {average_train_loss:.6f}")

In [ ]:
model.save('LCA_weight/LCA_640.360_78k.keras')
model.save('LCA_weight/LCA_640.360_78k.h5')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
import matplotlib.pyplot as plt

image_size = (360, 640)
test_image_path = '/home/aryan/sih23/Dataset/MASK_CUSTOM/38342.jpg'  

# Load and preprocess the test image
test_image = load_img(test_image_path, target_size=image_size)
test_image = img_to_array(test_image)
test_image = test_image / 255.0  

# Reshape the test image to match the model's input shape
test_image = np.expand_dims(test_image, axis=0)

# Use the model to make predictions
predicted_image = model.predict(test_image)
predicted_image = np.clip(predicted_image, 0.0, 1.0)


plt.figure(figsize=(20, 7))
original_image = load_img(test_image_path, target_size=image_size)

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(original_image)


predicted_image = predicted_image[0]  
plt.subplot(1, 2, 2)
plt.title('Predicted Image')
plt.imshow(predicted_image)

plt.show()
